In [1]:
from methods import *
import os, shutil

In [2]:
# We have percent_data/percentages/dataset/train_orig.txt

#load hyperparameters
sizes = [0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

folders_list =  ['percent_data/' + str(int(size*100)) for size in sizes]
#size_folders = ['size_data_f1/' + size for size in sizes]

#datasets
datasets = ['sst2']

#number of output classes
num_classes_list = [2, 2, 2, 6, 2]

#number of augmentations per original sentence
n_aug_list_dict = {'size_data_f1/1_tiny': [32, 32, 32, 32, 32], 
					'size_data_f1/2_small': [32, 32, 32, 32, 32],
					'size_data_f1/3_standard': [16, 16, 16, 16, 4],
					'size_data_f1/4_full': [16, 16, 16, 16, 4]}

if not os.path.isdir('size_data_f1'):
    os.mkdir('size_data_f1')
#number of words for input
input_size_list = [50, 50, 40, 25, 25]


# Number of words for input
input_size_list = [50,50,40,25,25]

#word2vec dictionary
huge_word2vec = 'word2vec/glove.840B.300d.txt'
word2vec_len = 300

In [3]:
def run_cnn(train_file, test_file, num_classes, percent_dataset):

    #initialize model
    model = build_cnn(input_size, word2vec_len, num_classes)

    #load data
    train_x, train_y = get_x_y(train_file, num_classes, word2vec_len, input_size, word2vec, percent_dataset)
    test_x, test_y = get_x_y(test_file, num_classes, word2vec_len, input_size, word2vec, 1)

    #implement early stopping
    callbacks = [EarlyStopping(monitor='val_loss', patience=3)]

    #train model
    model.fit(	train_x, 
                train_y, 
                epochs=20, 
                callbacks=callbacks,
                validation_split=0.1, 
                batch_size=1024, 
                shuffle=True, 
                verbose=1)
    #model.save('checkpoints/lol')
    #model = load_model('checkpoints/lol')

    #evaluate model
    y_pred = model.predict(test_x)
    test_y_cat = one_hot_to_categorical(test_y)
    y_pred_cat = one_hot_to_categorical(y_pred)
    acc = accuracy_score(test_y_cat, y_pred_cat)

    #clean memory???
    train_x, train_y, test_x, test_y, model = None, None, None, None, None
    gc.collect()

    #return the accuracy
    #print("data with shape:", train_x.shape, train_y.shape, 'train=', train_file, 'test=', test_file, 'with fraction', percent_dataset, 'had acc', acc)
    return acc

In [6]:
#for each method
a_method = 'weda' #### TOGGLE EDA/WEDA
tfidf_func = 4

writer_cnn = open('outputs_percent/cnn_' + a_method + '_' +'tfidf'+str(tfidf_func)+'_'+ get_now_str() + '.txt', 'w')

#for storing the performances
performances_cnn = {size_folder:[] for size_folder in folders_list}

#for each percentage dataset
for size_folder in folders_list:

    writer_cnn.write(size_folder + '\n')

    #get all six datasets
    dataset_folders = [size_folder + '/' + s for s in datasets]



    #for each dataset
    for i in range(len(dataset_folders)):

        #initialize all the variables
        dataset_folder = dataset_folders[i]
        dataset = datasets[i]
        num_classes = num_classes_list[i]
        input_size = input_size_list[i]
        word2vec_pickle = dataset + '/word2vec.p'
        word2vec = load_pickle(word2vec_pickle)

        

        train_path = dataset_folder + '/train_weda_pct_tfidf'+str(tfidf_func)+'.txt' #### TOGGLE EDA/WEDA
        test_path = dataset + '/test.txt'
        acc = run_cnn(train_path, test_path, num_classes, percent_dataset=1)
        print("cnn aug ", acc, dataset_folder)

        performances_cnn[size_folder].append(acc)
        """
        acc = run_rnn(train_path, test_path, num_classes, percent_dataset=1)
        performances_rnn[alpha].append(acc)
        print("rnn aug", acc)
        """

writer_cnn.write(str(performances_cnn) + '\n')
#writer_rnn.write(str(performances_rnn)+"\n")
for size_folder in performances_cnn:
    line = str(size_folder) + ' : ' + str(sum(performances_cnn[size_folder])/len(performances_cnn[size_folder]))
    writer_cnn.write(line + '\n')
    print(line)
"""
for alpha in performances_rnn:
    line = str(alpha) + ' : ' + str(sum(performances_rnn[alpha])/len(performances_rnn[alpha]))
    writer_rnn.write(line + '\n')
    print(line)
"""
print(performances_cnn)

writer_cnn.close()    


Epoch 1/20
1/1 [==============================] - 1s 589ms/step - loss: 0.6936 - accuracy: 0.5221 - val_loss: 0.6759 - val_accuracy: 0.8630
Epoch 2/20
1/1 [==============================] - 0s 147ms/step - loss: 0.6727 - accuracy: 0.8630 - val_loss: 0.6573 - val_accuracy: 0.9452
Epoch 3/20
1/1 [==============================] - 0s 173ms/step - loss: 0.6510 - accuracy: 0.9391 - val_loss: 0.6366 - val_accuracy: 0.9726
Epoch 4/20
1/1 [==============================] - 0s 164ms/step - loss: 0.6298 - accuracy: 0.9726 - val_loss: 0.6161 - val_accuracy: 0.9726
Epoch 5/20
1/1 [==============================] - 0s 156ms/step - loss: 0.6073 - accuracy: 0.9787 - val_loss: 0.5951 - val_accuracy: 0.9726
Epoch 6/20
1/1 [==============================] - 0s 155ms/step - loss: 0.5842 - accuracy: 0.9878 - val_loss: 0.5724 - val_accuracy: 0.9726
Epoch 7/20
1/1 [==============================] - 0s 156ms/step - loss: 0.5599 - accuracy: 0.9878 - val_loss: 0.5484 - val_accuracy: 0.9863
Epoch 8/20
1/1 [====

7/7 [==============================] - 1s 150ms/step - loss: 0.0030 - accuracy: 1.0000 - val_loss: 0.0383 - val_accuracy: 0.9905
Epoch 19/20
7/7 [==============================] - 1s 148ms/step - loss: 0.0025 - accuracy: 1.0000 - val_loss: 0.0379 - val_accuracy: 0.9905
Epoch 20/20
7/7 [==============================] - 1s 145ms/step - loss: 0.0021 - accuracy: 1.0000 - val_loss: 0.0367 - val_accuracy: 0.9905
cnn aug  0.8033161806746713 percent_data/10/sst2
Epoch 1/20
13/13 [==============================] - 3s 210ms/step - loss: 0.6532 - accuracy: 0.6388 - val_loss: 0.5301 - val_accuracy: 0.7828
Epoch 2/20
13/13 [==============================] - 2s 172ms/step - loss: 0.4780 - accuracy: 0.8133 - val_loss: 0.4090 - val_accuracy: 0.8261
Epoch 3/20
13/13 [==============================] - 2s 160ms/step - loss: 0.3526 - accuracy: 0.8604 - val_loss: 0.3232 - val_accuracy: 0.8762
Epoch 4/20
13/13 [==============================] - 2s 191ms/step - loss: 0.2457 - accuracy: 0.9186 - val_loss: 0.

26/26 [==============================] - 5s 182ms/step - loss: 6.7706e-04 - accuracy: 1.0000 - val_loss: 0.0452 - val_accuracy: 0.9834
Epoch 17/20
26/26 [==============================] - 5s 185ms/step - loss: 5.8767e-04 - accuracy: 1.0000 - val_loss: 0.0453 - val_accuracy: 0.9834
Epoch 18/20
26/26 [==============================] - 5s 178ms/step - loss: 5.0112e-04 - accuracy: 1.0000 - val_loss: 0.0448 - val_accuracy: 0.9834
Epoch 19/20
26/26 [==============================] - 5s 178ms/step - loss: 4.2523e-04 - accuracy: 1.0000 - val_loss: 0.0451 - val_accuracy: 0.9831
Epoch 20/20
26/26 [==============================] - 5s 178ms/step - loss: 3.9938e-04 - accuracy: 1.0000 - val_loss: 0.0447 - val_accuracy: 0.9834
cnn aug  0.8267581475128645 percent_data/40/sst2
Epoch 1/20
33/33 [==============================] - 7s 198ms/step - loss: 0.6280 - accuracy: 0.6599 - val_loss: 0.4372 - val_accuracy: 0.7979
Epoch 2/20
33/33 [==============================] - 6s 176ms/step - loss: 0.3840 - acc

Epoch 4/20
52/52 [==============================] - 10s 191ms/step - loss: 0.0793 - accuracy: 0.9859 - val_loss: 0.1040 - val_accuracy: 0.9658
Epoch 5/20
52/52 [==============================] - 9s 172ms/step - loss: 0.0327 - accuracy: 0.9967 - val_loss: 0.0845 - val_accuracy: 0.9718
Epoch 6/20
52/52 [==============================] - 8s 150ms/step - loss: 0.0153 - accuracy: 0.9989 - val_loss: 0.0768 - val_accuracy: 0.9738
Epoch 7/20
52/52 [==============================] - 8s 153ms/step - loss: 0.0084 - accuracy: 0.9996 - val_loss: 0.0718 - val_accuracy: 0.9758
Epoch 8/20
52/52 [==============================] - 8s 153ms/step - loss: 0.0053 - accuracy: 0.9997 - val_loss: 0.0698 - val_accuracy: 0.9767
Epoch 9/20
52/52 [==============================] - 9s 183ms/step - loss: 0.0035 - accuracy: 0.9999 - val_loss: 0.0682 - val_accuracy: 0.9767
Epoch 10/20
52/52 [==============================] - 12s 239ms/step - loss: 0.0025 - accuracy: 1.0000 - val_loss: 0.0683 - val_accuracy: 0.9770
Epo

In [ ]:
### For training on the original data, without augmentations
#for each method
a_method = 'no_aug' #### TOGGLE EDA/WEDA

writer_cnn = open('outputs_percent/cnn_' + a_method + '_' + get_now_str() + '.txt', 'w')

#for storing the performances
performances_cnn = {size_folder:[] for size_folder in folders_list}

#for each percentage dataset
for size_folder in folders_list:

    writer_cnn.write(size_folder + '\n')

    #get all six datasets
    dataset_folders = [size_folder + '/' + s for s in datasets]



    #for each dataset
    for i in range(len(dataset_folders)):

        #initialize all the variables
        dataset_folder = dataset_folders[i]
        dataset = datasets[i]
        num_classes = num_classes_list[i]
        input_size = input_size_list[i]
        word2vec_pickle = dataset + '/word2vec.p'
        word2vec = load_pickle(word2vec_pickle)

        

        train_path = dataset_folder + '/train_orig.txt' #### TOGGLE EDA/WEDA
        test_path = dataset + '/test.txt'
        acc = run_cnn(train_path, test_path, num_classes, percent_dataset=1)
        print("cnn aug ", acc, dataset_folder)

        performances_cnn[size_folder].append(acc)
        """
        acc = run_rnn(train_path, test_path, num_classes, percent_dataset=1)
        performances_rnn[alpha].append(acc)
        print("rnn aug", acc)
        """

writer_cnn.write(str(performances_cnn) + '\n')
#writer_rnn.write(str(performances_rnn)+"\n")
for size_folder in performances_cnn:
    line = str(size_folder) + ' : ' + str(sum(performances_cnn[size_folder])/len(performances_cnn[size_folder]))
    writer_cnn.write(line + '\n')
    print(line)
"""
for alpha in performances_rnn:
    line = str(alpha) + ' : ' + str(sum(performances_rnn[alpha])/len(performances_rnn[alpha]))
    writer_rnn.write(line + '\n')
    print(line)
"""
print(performances_cnn)

writer_cnn.close()    
